# Figshare API

In [ ]:
#import libraries
import requests as rq
import pandas as pd
from pprint import pprint as pp
import pickle

API access tokens have been stored in credentials.pkl file

In [ ]:
#import API access token
#at the moment, only have a single token in this credentials.pkl, which isn't the long term plan - need to change this
FIGSHARE_TOKEN = pickle.load(open('credentials.pkl', 'rb'))

In [ ]:
#Search seems to be all by type (articles, collections, etc.)
#All are public
#Plan is to search (1) articles, (2) collections, (3) projects, (4) data
#Article search may be minimally useful, but if have linked DOIs in object, could be linked data

#Can set up to cycle through which search (articles, collections, data) and by page

Note: code as written (7/1) does NOT handle pagination yet - only goes through the workflow for first page

Overall workflow: use main search to get IDs of matching objects, then use IDs to get full object details & associated files

## Specify search parameters

In [ ]:
#FIGSHARE_TOKEN imported from credentials.pkl
HEADERS = {'Authorization': 'token '+ FIGSHARE_TOKEN}

#Specify search terms
SEARCH = 'machine learning'

#Specify page to return with search (results are paginated)
PAGE = 1

#Specify number of results included on a page (default is 10, max is 1000)
PAGE_SIZE = 1000

#Having variable definition outside PARAMS so can put into loop later

#Specify page and page size parameters
#Other search options are available, including limit and offset, but at the moment page/page size seem most useful
    #if set both page/page size and limit/offset, get:
    #{'message': 'Pagination options can be set either via page/page_size or limit/offset params','code': 'ConflictingPaginationOptions'}
    
    #'limit': 1000, #Number of results included on a page. Used for pagination with query (optional) - not sure how differs from page_size
    #'offset': 1 #, #Where to start the listing(the offset of the first result). Used for pagination with limit (optional)
#We don't need to specify other search parameters like institution, group, modified since, etc.

#Specify which search (collections, articles, projects, data)
ARTICLE_URL = 'https://api.figshare.com/v2/articles'
COLLECTIONS_URL = "https://api.figshare.com/v2/collections/search"
PROJECTS_URL = "https://api.figshare.com/v2/projects/search"
#DATA_URL = ???

#Full search parameters
PARAMS = {
    'search_for': SEARCH, #search term
    'page': PAGE, 
    'page_size': PAGE_SIZE,  
    }

## Example: search public articles

In [ ]:
## Run search for public articles
response = rq.get(ARTICLE_URL, params=PARAMS, headers=HEADERS)

In [ ]:
## Put output into json format
output = response.json()

In [ ]:
## See what output looks like
output

In [ ]:
#Convert output to pd dataframe and see table format
df = pd.DataFrame(output)

df

In [ ]:
## Alternative approach - normalize output
## Same output as pd dataframe - thinking stick with pd for ease of data wrangling downstream
#df2 = pd.json_normalize(output)
#df2

In [ ]:
## Extract IDs
article_ids = list(df.id)

In [ ]:
article_ids

In [ ]:
## List files associated with public articles (by ID)

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}/files

#approach: loop through all IDs, collapse output to table, iterate and append next table

In [ ]:
## View article details

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}